In [1]:
#Import bibliotek
import pandas as pd
import csv as csv
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import scipy
#import sklearn
import Analysis
import mlxtend 
from mlxtend.frequent_patterns import apriori, association_rules
sns.set_theme(style="darkgrid")

c:\Users\Robin\anaconda3\lib\site-packages\seaborn\rcmod.py:400: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
c:\Users\Robin\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [2]:
#Zmienne konfiguracyjne
_SavePath = "C:\\Users\\Robin\\Jupyter Notebooks\\apropri\\Apropri.csv"
path_s_d = "C:\\Users\\Robin\\Jupyter Notebooks\\Dane\\Dane zebrane - Warszawa\\Warszawa.csv"

In [3]:
def get_rules(apriori_df, min_support=0.01):
    frequent_itemsets = apriori(apriori_df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=min_support)
    return rules
def get_columns(dfs:pd.DataFrame, label:str, frange:tuple, iloczyn:int, stala:int=0):
    krok = iloczyn    
    for x in range(frange[0],frange[1],frange[2]):
        left = x*iloczyn+stala
        right = (x+1)*iloczyn+stala
        dfs[label+'('+ str(left) + ','+str(right)+')'] = (dfs[label]>=left) & (dfs[label]<right)
    dfs.drop(labels=label,axis=1,inplace=True)

In [4]:
#Wczytanie plików .csv pandasem
_SavePath = "C:\\Users\\Robin\\Jupyter Notebooks\\apropri\\Apropri_Ulewa.csv"

df = pd.read_csv(filepath_or_buffer = path_s_d, delimiter = ";", encoding='cp1250', low_memory=False)#, index_col=[0])
Temp = np.zeros(20454)
Temp1 = Temp.copy()
Temp2 = Temp.copy()
for x in range(1,(int)(df.size / df.columns.size)):
    Temp[x] = df.iloc[x]['Srednie dobowe cisnienie na poziomie stacji [hPa]'] - df.iloc[x-1]['Srednie dobowe cisnienie na poziomie stacji [hPa]']
    Temp1[x] = df.iloc[x]['Srednia temperatura dobowa [oC]'] - df.iloc[x-1]['Srednia temperatura dobowa [oC]']
    Temp2[x] = df.iloc[x]['Minimalna temperatura dobowa [oC]'] - df.iloc[x-1]['Minimalna temperatura dobowa [oC]']

df['Minimalna temperatura dobowa [oC]']
df['dP'] = Temp
df['dTsr'] = Temp1
df['dTmin'] = Temp2
Ulewa = df.copy()

Ulewa = Ulewa.loc[(Ulewa['Rodzaj opadu [S/W/ ]'] == 'W') & (Ulewa['Status pomiaru SMDB'] != '8')]
Ulewa = Ulewa.loc[(Ulewa['Status pomiaru SMDB'] != '8')]
Ulewa = Ulewa.loc[(Ulewa['Status pomiaru PPPS'] != '8') & (Ulewa['Status pomiaru WLGS'] != '8')]
Ulewa = Ulewa.loc[(Ulewa['Status pomiaru STD'] != '8')]

apUlewa = pd.DataFrame()
Ulewa = Ulewa.loc[Ulewa['Suma dobowa opadu [mm]']>=30].copy()
apUlewa['Ulewa'] = Ulewa['Suma dobowa opadu [mm]'].copy()
apUlewa['Temp'] = Ulewa['Srednia temperatura dobowa [oC]'].copy()
apUlewa['Cisnienie'] = Ulewa['Srednie dobowe cisnienie na poziomie stacji [hPa]'].copy()
apUlewa['Wilgotnosc'] = Ulewa['Srednia dobowa wilgotnosc wzgledna [%]'].copy()
apUlewa['Wiatr>15'] = Ulewa['Czas trwania wiatru >15m/s [godziny]'].copy()
apUlewa['Wiatr>10'] = Ulewa['Czas trwania wiatru >=10m/s [godziny]'].copy()

apUlewa['Zima'] = (Ulewa['Miesiac'] == 12) | (Ulewa['Miesiac'] < 3)
apUlewa['Wiosna'] = (Ulewa['Miesiac'] > 2) & (Ulewa['Miesiac'] < 6)
apUlewa['Lato'] = (Ulewa['Miesiac'] > 5) & (Ulewa['Miesiac'] < 9)
apUlewa['Jesien'] = (Ulewa['Miesiac'] > 8) & (Ulewa['Miesiac'] < 12)
apUlewa.loc[apUlewa['Ulewa']<30,('Ulewa')] = False
apUlewa.loc[apUlewa['Ulewa']>=30,('Ulewa')] = True
#print(apUlewa.loc[apUlewa['Ulewa'] == False])
apUlewa
#print(apUlewa.loc[apUlewa['Ulewa']==False])
get_columns(apUlewa,'Temp',(-4,4,1),10,0)
get_columns(apUlewa,'Wilgotnosc',(0,10,1),10,0)
get_columns(apUlewa,'Cisnienie',(0,8,1),10,960)

apUlewa.loc[apUlewa['Wiatr>15']>0,('Wiatr>15')] = True
apUlewa.loc[apUlewa['Wiatr>15']==0,('Wiatr>15')] = False

apUlewa.loc[apUlewa['Wiatr>10']>0,('Wiatr>10')] = True
apUlewa.loc[apUlewa['Wiatr>10']==0,('Wiatr>10')] = False

#print('1488')
#print(apUlewa.loc[(apUlewa['Cisnienie4']) & (apUlewa['Temp2']) & (apUlewa['Lato']),('Temp2')])

apriori_rulz = get_rules(apUlewa,min_support=0.01)
apriori_rulz['length'] = apriori_rulz['antecedents'].apply(lambda x: len(x))
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].astype('string')
apriori_rulz['consequents'] = apriori_rulz['consequents'].astype('string')

apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removeprefix("frozenset({")
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removesuffix("})")

apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removeprefix("frozenset({")
apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removesuffix("})")


apriori_rulz = apriori_rulz.loc[(apriori_rulz['length'] > 0) & (apriori_rulz['support'] >= 0.01)]

print(apriori_rulz.loc[apriori_rulz['consequents'].str.contains('Ulewa')])

apriori_rulz.to_csv(path_or_buf=_SavePath, sep=';')
#print(apriori_rulz)
#Ulewa_rulz.to_csv(path_or_buf=_SavePath, sep=';')
#Ulewa_rulz.loc[Ulewa_rulz['consequents']=='(Wiatr10)']
#print(Ulewa_rulz)

               antecedents                                        consequents  \
1               'Wiatr>15'                                            'Ulewa'   
3               'Wiatr>10'                                            'Ulewa'   
5                 'Wiosna'                                            'Ulewa'   
7                   'Lato'                                            'Ulewa'   
9                 'Jesien'                                            'Ulewa'   
...                    ...                                                ...   
8000            'Wiatr>15'  'Wilgotnosc(80,90)', 'Wiatr>10', 'Cisnienie(98...   
8001            'Wiatr>10'  'Wilgotnosc(80,90)', 'Wiatr>15', 'Cisnienie(98...   
8002  'Cisnienie(980,990)'  'Wilgotnosc(80,90)', 'Wiatr>15', 'Wiatr>10', '...   
8003              'Jesien'  'Wilgotnosc(80,90)', 'Wiatr>15', 'Wiatr>10', '...   
8005         'Temp(10,20)'  'Wilgotnosc(80,90)', 'Wiatr>15', 'Wiatr>10', '...   

      antecedent support  c

c:\Users\Robin\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [5]:
_SavePath = "C:\\Users\\Robin\\Jupyter Notebooks\\apropri\\Apropri_Sniezyca.csv"
Sniezyca = df.copy()

#Sniezyca=Sniezyca.loc[Sniezyca['Wysokosc pokrywy snieznej [cm]'] >= 15]
#Sniezyca=Sniezyca.loc[Sniezyca['Czas trwania wiatru >=10m/s [godziny]'] >= 1]

#df['Czas trwania wiatru >=10m/s [godziny]']
Sniezyca = Sniezyca.loc[(Sniezyca['Rodzaj opadu [S/W/ ]'] == 'S') & (Sniezyca['Status pomiaru SMDB'] != '8')]
Sniezyca = Sniezyca.loc[(Sniezyca['Status pomiaru PPPS'] != '8') & (Sniezyca['Status pomiaru WLGS'] != '8')]
Sniezyca = Sniezyca.loc[(Sniezyca['Status pomiaru STD'] != '8')]
Sniezyca = Sniezyca.loc[(Sniezyca['Status pomiaru PKSN'] != '8')]
Sniezyca = Sniezyca.loc[Sniezyca['Wysokosc pokrywy snieznej [cm]']>=15].copy()
Sniezyca = Sniezyca.loc[Sniezyca['Czas trwania wiatru >=10m/s [godziny]'] >= 1].copy()
apSniezyca = pd.DataFrame()
#Sniezyca=Sniezyca.loc[Sniezyca['Suma dobowa opadu [mm]']>=30].copy()
apSniezyca['Opad'] = Sniezyca['Suma dobowa opadu [mm]'].copy()
apSniezyca['Temp'] = Sniezyca['Srednia temperatura dobowa [oC]'].copy()
apSniezyca['Cisnienie'] = Sniezyca['Srednie dobowe cisnienie na poziomie stacji [hPa]'].copy()
apSniezyca['Wilgotnosc'] = Sniezyca['Srednia dobowa wilgotnosc wzgledna [%]'].copy()
apSniezyca['Pokrywa'] = Sniezyca['Wysokosc pokrywy snieznej [cm]'].copy()
apSniezyca['Wiatr>15'] = Sniezyca['Czas trwania wiatru >15m/s [godziny]'].copy()
apSniezyca['Wiatr>10'] = Sniezyca['Czas trwania wiatru >=10m/s [godziny]'].copy()
apSniezyca['Zima'] = (Sniezyca['Miesiac'] == 12) | (Sniezyca['Miesiac'] < 3)
apSniezyca['Wiosna'] = (Sniezyca['Miesiac'] > 2) & (Sniezyca['Miesiac'] < 6)
apSniezyca['Lato'] = (Sniezyca['Miesiac'] > 5) & (Sniezyca['Miesiac'] < 9)
apSniezyca['Jesien'] = (Sniezyca['Miesiac'] > 8) & (Sniezyca['Miesiac'] < 12)
apSniezyca['Sniezyca'] = (apSniezyca['Pokrywa']>=15) & (apSniezyca['Wiatr>10']>=1)
#print(apSniezyca.loc[apSniezyca['Sniezyca'] == False])

get_columns(apSniezyca,'Opad',(0,4,1),10,0)
get_columns(apSniezyca,'Temp',(-4,4,1),10,0)
get_columns(apSniezyca,'Wilgotnosc',(0,10,1),10,0)
get_columns(apSniezyca,'Cisnienie',(0,8,1),10,960)
get_columns(apSniezyca,'Pokrywa',(0,8,1),10,0)

apSniezyca.loc[apSniezyca['Wiatr>15']>0,('Wiatr>15')] = True
apSniezyca.loc[apSniezyca['Wiatr>15']==0,('Wiatr>15')] = False

apSniezyca.loc[apSniezyca['Wiatr>10']>0,('Wiatr>10')] = True
apSniezyca.loc[apSniezyca['Wiatr>10']==0,('Wiatr>10')] = False
#apSniezyca
#apSniezyca[apSniezyca['Temp']==0]
#apSniezyca[apSniezyca['Cisnienie8'] == True]


apriori_rulz = get_rules(apSniezyca,min_support=0.01)
apriori_rulz['length'] = apriori_rulz['antecedents'].apply(lambda x: len(x))
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].astype('string')
apriori_rulz['consequents'] = apriori_rulz['consequents'].astype('string')

apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removeprefix("frozenset({")
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removesuffix("})")

apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removeprefix("frozenset({")
apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removesuffix("})")
print(apriori_rulz.loc[apriori_rulz['consequents'].str.contains('Sniezyca')])
#print(apriori_rulz)


apriori_rulz = apriori_rulz.loc[(apriori_rulz['length'] > 0) & (apriori_rulz['support'] >= 0.10)]
apriori_rulz.to_csv(path_or_buf=_SavePath, sep=';')
#apriori_rulz.loc[apriori_rulz['consequents']=='(Wiatr10)']
#print(apriori_rulz)

c:\Users\Robin\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


                 antecedents  \
5                 'Wiatr>15'   
33                'Wiatr>10'   
79                    'Zima'   
125                 'Wiosna'   
142             'Opad(0,10)'   
...                      ...   
187410  'Wilgotnosc(90,100)'   
187411          'Opad(0,10)'   
187413  'Cisnienie(980,990)'   
187414      'Pokrywa(20,30)'   
187415                'Zima'   

                                              consequents  antecedent support  \
5                                              'Sniezyca'            0.128205   
33                                             'Sniezyca'            1.000000   
79                                             'Sniezyca'            0.948718   
125                                            'Sniezyca'            0.051282   
142                                            'Sniezyca'            0.974359   
...                                                   ...                 ...   
187410  'Wiatr>15', 'Wiatr>10', 'Temp(0,10)', 'O

In [28]:
_SavePath = "C:\\Users\\Robin\\Jupyter Notebooks\\apropri\\Apropri_Wichura.csv"
Wichura = df.copy()

#Wichura=Wichura.loc[Wichura['Wysokosc pokrywy snieznej [cm]'] >= 15]
#Wichura=Wichura.loc[Wichura['Czas trwania wiatru >=15m/s [godziny]'] >= 0]

#df['Czas trwania wiatru >=10m/s [godziny]']
#Wichura = Wichura.loc[(Wichura['Czas trwania wiatru >15m/s [godziny]'] != 0) & (Wichura['Status pomiaru FF15'] != '8')]
Wichura = Wichura.loc[(Wichura['Status pomiaru PPPS'] != '8')]
Wichura = Wichura.loc[(Wichura['Status pomiaru STD'] != '8')]
Wichura = Wichura.loc[(Wichura['Czas trwania wiatru >15m/s [godziny]']>0)].copy()
apWichura = pd.DataFrame()

apWichura['Temp'] = Wichura['Srednia temperatura dobowa [oC]'].copy()
apWichura['Cisnienie'] = Wichura['Srednie dobowe cisnienie na poziomie stacji [hPa]'].copy()
apWichura['Srwiatr'] = Wichura['Srednia dobowa predkosc wiatru [m/s]'].copy()
apWichura['Wichura'] = Wichura['Czas trwania wiatru >15m/s [godziny]'].copy()
apWichura['Zima'] = (Wichura['Miesiac'] == 12) | (Wichura['Miesiac'] < 3)
apWichura['Wiosna'] = (Wichura['Miesiac'] > 2) & (Wichura['Miesiac'] < 6)
apWichura['Lato'] = (Wichura['Miesiac'] > 5) & (Wichura['Miesiac'] < 9)
apWichura['Jesien'] = (Wichura['Miesiac'] > 8) & (Wichura['Miesiac'] < 12)
apWichura['Zachmurzenie'] = Wichura['Srednie dobowe zachmurzenie ogolne [oktanty]'].copy()
apWichura['Wilgotnosc']=Wichura['Srednia dobowa wilgotnosc wzgledna [%]'].copy()
apWichura['dP'] = Wichura['dP'].copy()
apWichura['dTsr'] = Wichura['dTsr'].copy()
apWichura['dTmin'] = Wichura['dTmin'].copy()

apWichura.loc[apWichura['Wichura']>0,'Wichura'] = True 
apWichura.loc[apWichura['Wichura']==0,'Wichura'] = False 
#print(apWichura.loc[apWichura['Wichura'] == False])
get_columns(apWichura,'Temp',(-4,4,1),10,0)
get_columns(apWichura,'Srwiatr',(0,7,1),2,0)
get_columns(apWichura,'Cisnienie',(0,8,1),10,960)

get_columns(apWichura,'dP',(-3,3,1),10,0) #-28,8-25,8
get_columns(apWichura,'dTsr',(-3,3,1),5,0) # -12,13.5
get_columns(apWichura,'dTmin',(-3,3,1),5,0) #-12.4, 15
get_columns(apWichura,'Zachmurzenie',(0,8,1),1,0)
get_columns(apWichura,'Wilgotnosc',(0,10,1),10,0)
apWichura.loc[apWichura['Wichura']>0,('Wichura')] = True
apWichura.loc[apWichura['Wichura']==0,('Wichura')] = False
#print(apWichura.tail())

apriori_rulz = get_rules(apWichura,min_support=0.10)
apriori_rulz['length'] = apriori_rulz['antecedents'].apply(lambda x: len(x))
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].astype('string')
apriori_rulz['consequents'] = apriori_rulz['consequents'].astype('string')

apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removeprefix("frozenset({")
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removesuffix("})")

apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removeprefix("frozenset({")
apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removesuffix("})")
print(apriori_rulz.loc[apriori_rulz['consequents'].str.contains('Wichura')])
print(apriori_rulz.loc[apriori_rulz['antecedents'].str.contains('Wichura')])
#print(apriori_rulz)

apriori_rulz = apriori_rulz.loc[(apriori_rulz['length'] > 0) & (apriori_rulz['support'] >= 0.000001)]
apriori_rulz.to_csv(path_or_buf=_SavePath, sep=';')
#apriori_rulz.loc[apriori_rulz['consequents']=='(Wiatr10)']
#print(apriori_rulz)

                     antecedents  \
1                         'Zima'   
3                       'Wiosna'   
5                         'Lato'   
7                       'Jesien'   
9                  'Temp(-10,0)'   
...                          ...   
4284  'Temp(0,10)', 'dTmin(0,5)'   
4285         'Wilgotnosc(80,90)'   
4287                 'dTsr(0,5)'   
4288                'Temp(0,10)'   
4289                'dTmin(0,5)'   

                                            consequents  antecedent support  \
1                                             'Wichura'            0.427350   
3                                             'Wichura'            0.256410   
5                                             'Wichura'            0.119658   
7                                             'Wichura'            0.196581   
9                                             'Wichura'            0.119658   
...                                                 ...                 ...   
4284        'W

c:\Users\Robin\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [7]:
_SavePath = "C:\\Users\\Robin\\Jupyter Notebooks\\apropri\\Apropri_Burza.csv"
Burza = df.copy()

#df['Czas trwania wiatru >=10m/s [godziny]']
Burza = Burza.loc[(Burza['Status pomiaru BRZA'] != '8')]
Burza = Burza.loc[(Burza['Status pomiaru PPPS'] != '8')]
Burza = Burza.loc[(Burza['Status pomiaru STD'] != '8')]
Burza = Burza.loc[(Burza['Status pomiaru DZBL'] != '8')]
Burza = Burza.loc[(Burza['Czas trwania burzy  [godziny]'] > 0)]
apBurza = pd.DataFrame()
#Burza=Burza.loc[Burza['Suma dobowa opadu [mm]']>=30].copy()

apBurza['Opad'] = Burza['Suma dobowa opadu [mm]'].copy()
apBurza['Temp'] = Burza['Srednia temperatura dobowa [oC]'].copy()
apBurza['Cisnienie'] = Burza['Srednie dobowe cisnienie na poziomie stacji [hPa]'].copy()
apBurza['Wilgotnosc'] = Burza['Srednia dobowa wilgotnosc wzgledna [%]'].copy()
apBurza['Srwiatr'] = Burza['Srednia dobowa predkosc wiatru [m/s]']
apBurza['Blysk'] = Burza['Wystapienie blyskawicy  [0/1]']
apBurza['Burza'] = Burza['Czas trwania burzy  [godziny]']>0
apBurza['Zima'] = (Burza['Miesiac'] == 12) | (Burza['Miesiac'] < 3)
apBurza['Wiosna'] = (Burza['Miesiac'] > 2) & (Burza['Miesiac'] < 6)
apBurza['Lato'] = (Burza['Miesiac'] > 5) & (Burza['Miesiac'] < 9)
apBurza['Jesien'] = (Burza['Miesiac'] > 8) & (Burza['Miesiac'] < 12)

print(apBurza.loc[apBurza['Burza'] == False])
get_columns(apBurza,'Opad',(0,7,1),10,0)
get_columns(apBurza,'Temp',(-4,4,1),10,0)
get_columns(apBurza,'Wilgotnosc',(0,10,1),10,0)
get_columns(apBurza,'Cisnienie',(0,8,1),10,960)
get_columns(apBurza,'Srwiatr',(0,7,1),2,0)

apBurza['Blysk'] = apBurza['Blysk'] == 1

apriori_rulz = get_rules(apBurza,min_support=0.01)
apriori_rulz['length'] = apriori_rulz['antecedents'].apply(lambda x: len(x))
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].astype('string')
apriori_rulz['consequents'] = apriori_rulz['consequents'].astype('string')

apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removeprefix("frozenset({")
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removesuffix("})")

apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removeprefix("frozenset({")
apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removesuffix("})")
print(apriori_rulz.loc[apriori_rulz['consequents'].str.contains('Burza')])
#print(apriori_rulz)
apriori_rulz = apriori_rulz.loc[(apriori_rulz['length'] > 0) & (apriori_rulz['support'] >= 0.05)]
apriori_rulz.to_csv(path_or_buf=_SavePath, sep=';')
#apriori_rulz.loc[apriori_rulz['consequents']=='(Wiatr10)']
#print(apriori_rulz)

Empty DataFrame
Columns: [Opad, Temp, Cisnienie, Wilgotnosc, Srwiatr, Blysk, Burza, Zima, Wiosna, Lato, Jesien]
Index: []
                 antecedents  \
1                    'Blysk'   
27                    'Zima'   
29                  'Wiosna'   
31                    'Lato'   
33                  'Jesien'   
...                      ...   
42757         'Srwiatr(2,4)'   
42758           'Opad(0,10)'   
42759  'Cisnienie(990,1000)'   
42760               'Jesien'   
42761          'Temp(10,20)'   

                                             consequents  antecedent support  \
1                                                'Burza'            0.056119   
27                                               'Burza'            0.014199   
29                                               'Burza'            0.258959   
31                                               'Burza'            0.648411   
33                                               'Burza'            0.078431   
...          

In [16]:
_SavePath = "C:\\Users\\Robin\\Jupyter Notebooks\\apropri\\Apropri_Upal.csv"
Upal = df.copy()

Upal = Upal.loc[(Upal['Status pomiaru TMAX'] != '8') & (Upal['Status pomiaru FWS'] != '8') & (Upal['Status pomiaru TMIN'] != '8')]
Upal = Upal.loc[(Upal['Status pomiaru PPPS'] != '8') & (Upal['Status pomiaru NOS'] != '8')]
Upal = Upal.loc[(Upal['Maksymalna temperatura dobowa [oC]'] >= 30) & (Upal['Minimalna temperatura dobowa [oC]'] >= 15)]

apUpal = pd.DataFrame()

apUpal['Cisnienie'] = Upal['Srednie dobowe cisnienie na poziomie stacji [hPa]'].copy()
apUpal['Srwiatr'] = Upal['Srednia dobowa predkosc wiatru [m/s]']
apUpal['Tempmax'] = Upal['Maksymalna temperatura dobowa [oC]']
apUpal['Zachmurzenie'] = Upal['Srednie dobowe zachmurzenie ogolne [oktanty]']
apUpal['Upal'] = (Upal['Maksymalna temperatura dobowa [oC]'] >= 30) & (Upal['Minimalna temperatura dobowa [oC]'] >= 15)
apUpal['Zima'] = (Upal['Miesiac'] == 12) | (Upal['Miesiac'] < 3)
apUpal['Wiosna'] = (Upal['Miesiac'] > 2) & (Upal['Miesiac'] < 6)
apUpal['Lato'] = (Upal['Miesiac'] > 5) & (Upal['Miesiac'] < 9)
apUpal['Jesien'] = (Upal['Miesiac'] > 8) & (Upal['Miesiac'] < 12)
#print(apUpal.loc[apUpal['Upal'] == False])
get_columns(apUpal,'Tempmax',(-4,4,1),10,0)
get_columns(apUpal,'Srwiatr',(0,7,1),2,0)
get_columns(apUpal,'Cisnienie',(0,8,1),10,960)
get_columns(apUpal,'Zachmurzenie',(0,9,1),1,0)

apriori_rulz = get_rules(apUpal,min_support=0.01)
apriori_rulz['length'] = apriori_rulz['antecedents'].apply(lambda x: len(x))
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].astype('string')
apriori_rulz['consequents'] = apriori_rulz['consequents'].astype('string')

apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removeprefix("frozenset({")
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removesuffix("})")

apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removeprefix("frozenset({")
apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removesuffix("})")
print(apriori_rulz.loc[apriori_rulz['consequents'].str.contains('Upal')])
apriori_rulz = apriori_rulz.loc[(apriori_rulz['length'] > 0) & (apriori_rulz['support'] >= 0.01)]
#print(apriori_rulz)
apriori_rulz.to_csv(path_or_buf=_SavePath, sep=';')
#apriori_rulz.loc[apriori_rulz['consequents']=='(Wiatr10)']
#print(apriori_rulz)

                 antecedents  \
1                   'Wiosna'   
3                     'Lato'   
5           'Tempmax(30,40)'   
7             'Srwiatr(0,2)'   
9             'Srwiatr(2,4)'   
...                      ...   
8240     'Zachmurzenie(4,5)'   
8242        'Tempmax(30,40)'   
8243                  'Lato'   
8244          'Srwiatr(4,6)'   
8245  'Cisnienie(1000,1010)'   

                                            consequents  antecedent support  \
1                                                'Upal'            0.028754   
3                                                'Upal'            0.961661   
5                                                'Upal'            1.000000   
7                                                'Upal'            0.153355   
9                                                'Upal'            0.661342   
...                                                 ...                 ...   
8240  'Upal', 'Tempmax(30,40)', 'Lato', 'Srwiatr(4,6...       

In [9]:
_SavePath = "C:\\Users\\Robin\\Jupyter Notebooks\\apropri\\Apropri_Mroz.csv"
Mroz = df.copy()

#Mroz=Mroz.loc[Mroz['Wysokosc pokrywy snieznej [cm]'] >= 15]
#Mroz=Mroz.loc[Mroz['Czas trwania wiatru >=10m/s [godziny]'] >= 1]

#df['Czas trwania wiatru >=10m/s [godziny]']
Mroz = Mroz.loc[(Mroz['Status pomiaru TMAX'] != '8') & (Mroz['Status pomiaru FWS'] != '8') & (Mroz['Status pomiaru TMIN'] != '8')]
Mroz = Mroz.loc[(Mroz['Status pomiaru PPPS'] != '8') & (Mroz['Status pomiaru NOS'] != '8')]
Mroz = Mroz.loc[(Mroz['Maksymalna temperatura dobowa [oC]'] <= -5) & (Mroz['Minimalna temperatura dobowa [oC]'] <= -15)]

apMroz = pd.DataFrame()
#Mroz=Mroz.loc[Mroz['Suma dobowa opadu [mm]']>=30].copy()

apMroz['Cisnienie'] = Mroz['Srednie dobowe cisnienie na poziomie stacji [hPa]'].copy()
apMroz['Srwiatr'] = Mroz['Srednia dobowa predkosc wiatru [m/s]']
apMroz['Tempmin'] = Mroz['Minimalna temperatura dobowa [oC]']
apMroz['Zachmurzenie'] = Mroz['Srednie dobowe zachmurzenie ogolne [oktanty]']
apMroz['Mroz'] = (Mroz['Maksymalna temperatura dobowa [oC]'] <= -5) & (Mroz['Minimalna temperatura dobowa [oC]'] <= -15)
apMroz['Zima'] = (Mroz['Miesiac'] == 12) | (Mroz['Miesiac'] < 3)
apMroz['Wiosna'] = (Mroz['Miesiac'] > 2) & (Mroz['Miesiac'] < 6)
apMroz['Lato'] = (Mroz['Miesiac'] > 5) & (Mroz['Miesiac'] < 9)
apMroz['Jesien'] = (Mroz['Miesiac'] > 8) & (Mroz['Miesiac'] < 12)

#print(apMroz.loc[apMroz['Mroz'] == False])
get_columns(apMroz,'Tempmin',(-4,4,1),10,0)
get_columns(apMroz,'Srwiatr',(0,7,1),2,0)
get_columns(apMroz,'Cisnienie',(0,8,1),10,960)
get_columns(apMroz,'Zachmurzenie',(0,9,1),1,0)

apriori_rulz = get_rules(apMroz,min_support=0.01)
apriori_rulz['length'] = apriori_rulz['antecedents'].apply(lambda x: len(x))
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].astype('string')
apriori_rulz['consequents'] = apriori_rulz['consequents'].astype('string')

apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removeprefix("frozenset({")
apriori_rulz['antecedents'] = apriori_rulz['antecedents'].str.removesuffix("})")

apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removeprefix("frozenset({")
apriori_rulz['consequents'] = apriori_rulz['consequents'].str.removesuffix("})")
print(apriori_rulz.loc[apriori_rulz['consequents'].str.contains('Mroz')])
#print(apriori_rulz)
apriori_rulz = apriori_rulz.loc[(apriori_rulz['length'] > 0) & (apriori_rulz['support'] >= 0.01)]
apriori_rulz.to_csv(path_or_buf=_SavePath, sep=';')
#apriori_rulz.loc[apriori_rulz['consequents']=='(Wiatr10)']
#print(apriori_rulz)

                  antecedents  \
1                      'Zima'   
3                    'Wiosna'   
5          'Tempmin(-30,-20)'   
7          'Tempmin(-20,-10)'   
9              'Srwiatr(0,2)'   
...                       ...   
13951  'Cisnienie(1020,1030)'   
13952      'Tempmin(-20,-10)'   
13953     'Zachmurzenie(2,3)'   
13954          'Srwiatr(4,6)'   
13955                  'Zima'   

                                             consequents  antecedent support  \
1                                                 'Mroz'            0.971311   
3                                                 'Mroz'            0.024590   
5                                                 'Mroz'            0.303279   
7                                                 'Mroz'            0.692623   
9                                                 'Mroz'            0.254098   
...                                                  ...                 ...   
13951  'Mroz', 'Tempmin(-20,-10)', 'Zachmur